In [1]:
import datetime

import os


os.getcwd()
if '.git' not in os.listdir():
    os.chdir('./../../../../')
os.listdir()

from ml.ml_core.src.core.model_factory import model_factory

import fasttext
import keras_core as keras
import numpy as np
import tensorflow as tf
from keras_core.models import load_model
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp
import tensorflow_hub as hub
import tensorflow_text as text


Using TensorFlow backend


2024-07-02 18:56:47.639384: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 18:56:47.639451: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 18:56:47.639475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 18:56:47.647471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Configurations

In [2]:
params_grid = {
    "model": "convnet",
    "embedder": "camembert",
    "embedding_data": "almanach-camembert-base",
    "dataset": "fr_hf.csv",
    "batch_size": 500
}

In [3]:
data_folder_src = 'ml/ml_core/data/processed/'
ml_core_folder_path = 'ml/ml_core/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{params_grid["dataset"]}_{date_str}'

log_dir = f"{ml_core_folder_path}log/fit/{experiment_name}"
model_weight_dst = f'{ml_core_folder_path}model_weights/{experiment_name}-model.keras'

In [4]:

vector_path = f'{ml_core_folder_path}embedded_vector/{params_grid["embedding_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'

# loading dataset

In [5]:
X = np.expand_dims(np.load(vector_x_dst), axis=1)
y = np.load(vector_y_dst)
X

array([[[ 0.04516848,  0.00071256,  0.03566859, ..., -0.09981904,
          0.03075864,  0.00900592]],

       [[ 0.01784509,  0.02990518,  0.07172755, ..., -0.08148439,
          0.05451308, -0.02512601]],

       [[ 0.03016657, -0.02432093,  0.12849346, ..., -0.07501146,
          0.06209692, -0.0390623 ]],

       ...,

       [[-0.07429342,  0.10049191,  0.09828879, ..., -0.06469899,
         -0.02038792, -0.01266516]],

       [[-0.06795738,  0.24448358,  0.12595129, ..., -0.06591158,
         -0.03617626, -0.05148328]],

       [[-0.0298334 ,  0.15611339,  0.14951393, ..., -0.09735298,
         -0.0018647 , -0.03475789]]], dtype=float32)

# Model

## callbacks

In [6]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=30,
    min_delta=0.0001,
    restore_best_weights=True,
    monitor='loss'
)


In [7]:

class EarlyStoppingLogging(keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [8]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

2024-07-02 18:56:49.795980: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 18:56:49.819835: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 18:56:49.819870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 18:56:49.822113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 18:56:49.822146: I tensorflow/compile

In [9]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

### training the model

In [10]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


model = model_factory(params_grid['model'], X_train.shape, 1)
print(model.summary)

(16263, 1, 768) 1
<bound method Model.summary of <Functional name=functional_1, built=True>>


In [11]:
model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

Epoch 1/500


2024-07-02 18:56:52.349744: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xbcecc10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-02 18:56:52.349784: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-02 18:56:52.385032: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-02 18:56:52.692733: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


25/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7413 - loss: 0.6147

2024-07-02 18:56:57.336190: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


33/33 ━━━━━━━━━━━━━━━━━━━━ 13s 195ms/step - accuracy: 0.7449 - loss: 0.6043 - val_accuracy: 0.7575 - val_loss: 0.5466
Epoch 2/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7600 - loss: 0.5310 - val_accuracy: 0.7575 - val_loss: 0.4954
Epoch 3/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7614 - loss: 0.4827 - val_accuracy: 0.7575 - val_loss: 0.4485
Epoch 4/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7634 - loss: 0.4649 - val_accuracy: 0.7940 - val_loss: 0.4383
Epoch 5/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7964 - loss: 0.4441 - val_accuracy: 0.8062 - val_loss: 0.4243
Epoch 6/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8073 - loss: 0.4289 - val_accuracy: 0.8120 - val_loss: 0.4162
Epoch 7/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8055 - loss: 0.4230 - val_accuracy: 0.8069 - val_loss: 0.4216
Epoch 8/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8135 - loss: 0.4138 - val_accuracy: 0.8195 - val_loss: 

# save model

In [12]:
model.save(model_weight_dst)